# Boltzmann Machines
relevant links  
https://www.cs.toronto.edu/~hinton/csc321/readings/boltz321.pdf  
https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf  
http://yann.lecun.com/exdb/publis/pdf/lecun-06.pdf  
https://en.wikipedia.org/wiki/Boltzmann_distribution  
https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15  
https://skymind.ai/wiki/restricted-boltzmann-machine  

use movie ratings dataset  
https://grouplens.org/datasets/movielens/  
http://files.grouplens.org/datasets/movielens/ml-1m-README.txt

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [1]:
class RBM(object):
    """
        Description:
            RBM based neural network
        
        functions:
            __init__(self, num_visible, num_hidden)
            sample_hidden
            
        
        variables:
            num_visible (nodes)
            num_hidden (nodes)
            
    """
    def __init__(self, num_visible, num_hidden):
        """
            Description:
                initialize weights and bias
            
            Parameters:
                num_visible: number of visible nodes
                num_hidden:  number of hidden nodes
                
            Returns:
                None
        """
        self.weights = torch.randn(num_hidden, num_visible)
        self.alpha   = torch.randn(1, num_hidden)
        self.beta    = torch.randn(1, num_visible)
        
    def sample_hidden(self, visible):
        """
            Description:
                calculates the probability the hidden node will
                be activated
            
            Parameters:
                visible: <vector> set of visible node values
                
            Returns:
                P_hidden: <vector> probability of the hidden
                    nodes being activated 
                D_hidden: <vector> sampled distribution (0s & 1s)
                
        """
        weight   = torch.mm(visible, self.weights.t())
        activate = weight + self.alpha.expand_as(weight)
        P_hidden = torch.sigmoid(activation)
        D_hidden = torch.bernoulli(P_hidden)
        
        return P_hidden, D_hidden
        
    def sample_visible(self, hidden):
        """
            Description:
                calculates the probability the visible node will
                be activated
            
            Parameters:
                hidden: <vector> set of hidden node values
                
            Returns:
                P_visible: <vector> probability of the visible
                    nodes being activated 
                D_visible: <vector> sampled distribution (0s & 1s)
        """
        weight    = torch.mm(hidden, self.weights)
        activate  = weight + self.beta.expand_as(weight)
        P_visible = torch.sigmoid(activation)
        D_visible = torch.bernoulli(P_visible)
        
        return P_visible, D_visible
    
    def train(self, input_nodes, P_input_nodes, k_nodes, P_k_nodes):
        """
            Description:
                Implemented contrastive divergence to approximate the 
                log likelihood gradients
                
            Parameters:
                input_nodes:   <vector>
                P_input_nodes: <vector>
                k_nodes:       <vector>
                P_k_nodes:     <vector>
        """
        

get the dataset

In [2]:
movies = pd.read_csv('Boltzmann_Machines/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users = pd.read_csv('Boltzmann_Machines/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings = pd.read_csv('Boltzmann_Machines/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
# UserID::MovieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


train-test set prep

In [20]:
X_train = pd.read_csv('Boltzmann_Machines/ml-100k/u1.base', delimiter='\t')
X_test  = pd.read_csv('Boltzmann_Machines/ml-100k/u1.test', delimiter='\t')

X_train = np.array(X_train, dtype='int')
X_test  = np.array(X_test, dtype='int')

# UserID::MovieID::Rating::Timestamp
X_test[0:5]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883]])

In [13]:
user_count  = int(max(max(X_train[:,0]), max(X_test[:,0])))
movie_count = int(max(max(X_train[:,1]), max(X_test[:,1])))
print("user_count:  ", user_count)
print("movie_count: ", movie_count)

user_count:   943
movie_count:  1682


transform data into matrix with users in rows and movies in columns

In [14]:
def convert(X, M, N):
    """
        converts into list of list for pytorch input
               : { movie 1, movie 2, ... , movie N}
        {
         user 1    rating   rating         rating
         user 2    rating   rating         rating
         ...
         user M    rating   rating         rating
         }
    """
    X_new = []
    
    for user_id in range(1, M+1):
        movies  = X[:,1][X[:,0] == user_id]
        ratings = X[:,2][X[:,0] == user_id]
        
        user_ratings = np.ones(N)*-1
        user_ratings[movies-1] = ratings
        X_new.append(list(user_ratings))
    
    return X_new

In [21]:
X_train = convert(X_train, user_count, movie_count)
X_test  = convert(X_test, user_count, movie_count)

Create torch tensors  
convert ratings into binary 1:like 0:dislike

In [22]:
X_trainT = torch.FloatTensor(X_train)
X_testT  = torch.FloatTensor(X_test)

X_trainT[X_trainT == 1] = 0
X_trainT[X_trainT == 2] = 0
X_trainT[X_trainT >= 3] = 1

X_testT[X_testT == 1] = 0
X_testT[X_testT == 2] = 0
X_testT[X_testT >= 3] = 1


Build the RBM